## Large Scale Data Mining assignment 1
### Frequent Itemsets and Association Rules
### 1.1. Apriori Algorithm implementation

Francisco Marques 97639 - Mestrado em Ciência de Dados
franciscocmarques@ua.pt

In [64]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext(appName="Assignment1 - Conditions")
print(sc.version)

3.3.2


In [3]:
spark = SparkSession.builder.appName("Assignment1 - Conditions").getOrCreate()
spark

Load file as Spark DataFrame. With printSchema() we can see the different types in this file. Proved by printing the dataframe, which shows the first few rows

In [6]:
file = spark.read.option('header','true').csv("data/conditions.csv.gz")
file.printSchema()

DataFrame[START: string, STOP: string, PATIENT: string, ENCOUNTER: string, CODE: string, DESCRIPTION: string]
root
 |-- START: string (nullable = true)
 |-- STOP: string (nullable = true)
 |-- PATIENT: string (nullable = true)
 |-- ENCOUNTER: string (nullable = true)
 |-- CODE: string (nullable = true)
 |-- DESCRIPTION: string (nullable = true)



Here we save the codes and respective descriptions for when we save the results to a file.

In [7]:
code_to_desc = dict(file.select('CODE', 'DESCRIPTION').distinct().collect())

First we select the columns with patient ID and condition code, and remove any duplicates, this is, we assume for this that people can only have a given condition one time, even if they're diagnosed multiple times. After this, we order the conditions' codes. This will help when we're searching for the frequent itemsets.

Then we create the first frequent itemsets RDD, for $k=1$, although this is simply to help us in finding for $k=2,3$.

In [8]:
from operator import add # equivalent to lambda x, y : x + y

support_threshold = 1000 # assignment support threshold value

# cleaned file RDD
patient_rdd = (file.select('PATIENT', 'CODE') 
                .distinct()
                .rdd.map(lambda row: (row[0], int(row[1]))) # convert DataFrame to RDD with records saved as tuples
                .groupByKey() # group values (codes) by patient
                .mapValues(tuple) # turn groupByKey iterable to tuple
                .map(lambda item: (item[0], sorted(item[1], reverse = True))) # sort codes by decreasing order
                )

# k = 1 frequent k-sets RDD
frequent_items = (patient_rdd.flatMap(lambda item: item[1]) # apply flatmap to the codes
                .map(lambda item: (item, 1)) # convert each code to (code, 1)
                .reduceByKey(add) # count codes
                .filter(lambda x: x[1] >= support_threshold) # remove codes that are not frequent, accordingly to threshold
                .sortBy(lambda x: -x[1]) # sort from most frequent to least frequent
                )

print('10 most frequent conditions')
print(frequent_items.take(10))

# converts (code, count) to code,  and transforms collected list to set for faster lookup
frequent_items_set = set(frequent_items.map(lambda item: item[0]).collect()) 

10 most frequent conditions
[(444814009, 751940), (195662009, 524692), (10509002, 461495), (162864005, 365567), (271737000, 355372), (15777000, 354315), (59621000, 305134), (40055000, 250239), (72892002, 205390), (19169002, 201894)]


Compare the number of records before and after grouping them by patient. For this we assumed that each patient can only have the same disease once.

In [9]:
print(f'Number of records in original dataframe: {file.rdd.count()}')
print(f'Number of records after removing duplicate diagnosis: {patient_rdd.count()}')

Number of records in original dataframe: 8493211
Number of records after removing duplicate diagnosis: 1157578


### $k=2$ frequent k-itemsets
Now we find the 10 most frequent pairs of conditions, i.e., pairs. We will make use of itertools' combinations functions to get every possible combination. This algorithm discards candidate pairs that aren't made up of only frequent items.

In [10]:
from itertools import combinations

# create combinations of the codes of each patient and keeps pairs that are in the frequent set of k = 1
# as these combinations are of 2 elements, patients with only 1 code are not considered.
frequent_pairs = (patient_rdd.flatMap(lambda item: [pair for pair in combinations(item[1] , 2) 
                if all(value in frequent_items_set for value in pair)]) # verifies if both elements in the pair are frequent
                .map(lambda item: (item, 1))
                .reduceByKey(add) # counts items
                .filter(lambda item: item[1] >= support_threshold) # removes non-frequent pairs
                .sortBy(lambda item: -item[1]) # sorts from most frequent to least
                )

print("10 most frequent pairs (k = 2 sets)")
most_frequent_pairs = frequent_pairs.take(10)
print(most_frequent_pairs)

# same as above
frequent_pairs_set = set(frequent_pairs.map(lambda item: item[0]).collect())

10 most frequent pairs (k = 2 sets)
[((444814009, 195662009), 343651), ((444814009, 10509002), 302516), ((271737000, 15777000), 289176), ((444814009, 162864005), 243812), ((444814009, 271737000), 236847), ((444814009, 15777000), 236320), ((195662009, 10509002), 211065), ((444814009, 59621000), 203450), ((195662009, 162864005), 167438), ((444814009, 40055000), 165530)]


### $k=3$
And the same for the triplets. Now we make use of the frequent pairs set to verify if the triplets' subsets are frequent and keep triplets that verify this condition for every combination.

In [11]:
frequent_triplets = (patient_rdd.flatMap(lambda item: [triplet for triplet in combinations(item[1], 3) 
                    if all(subset in frequent_pairs_set for subset in combinations(triplet, 2))])
                    # verifies if every subset of length 2 of the triplet is frequent;
                    # discards triplets with at least 1 subset that doesn't verify this
                    .map(lambda item: (item, 1)) # same as above
                    .reduceByKey(add)
                    .filter(lambda item: item[1] >= support_threshold)
                    .sortBy(lambda item: -item[1])
                    )

print("10 most frequent triplets (k = 3 sets)")
most_frequent_triplets = frequent_triplets.take(10)
print(most_frequent_triplets)

frequent_triplet_set = set(frequent_triplets.map(lambda item: item[0]).collect())

10 most frequent triplets (k = 3 sets)
[((444814009, 271737000, 15777000), 192819), ((444814009, 195662009, 10509002), 139174), ((271737000, 195662009, 15777000), 132583), ((271737000, 15777000, 10509002), 115510), ((444814009, 195662009, 162864005), 111860), ((444814009, 271737000, 195662009), 108560), ((444814009, 195662009, 15777000), 108083), ((271737000, 59621000, 15777000), 99818), ((444814009, 162864005, 10509002), 97384), ((444814009, 271737000, 10509002), 94793)]


#### 1.2 Association Rules generation
Now we want to get every and any association rule $\{X\}\rightarrow\{Y\}$ and $\{X,Y\}\rightarrow\{Z\}$ with a minimum standardized lift of $0.2$. This means that we want to obtain the association rules using the frequent pairs, and then using the frequent triplets. 

First we count the number of frequent items, *total_singles*; pairs, *total_pairs*; and triplets, *total_triplets*.

We can see that we have many more triplets than items or pairs.

In [12]:
# Simple implementation of counting the total items, pairs and triplets

total_singles = sum(frequent_items.map(lambda item: item[1]).collect())

total_pairs = sum(frequent_pairs.map(lambda item: item[1]).collect())

total_triplets = sum(frequent_triplets.map(lambda item: item[1]).collect())

print(f'Total single items: {total_singles}')
print(f'Total pairs: {total_pairs}')
print(f'Total triplets: {total_triplets}')

Total single items: 7290639
Total pairs: 24939331
Total triplets: 52773489


Set a standardized lift threshold and define the function to compute it, accordingly to the class slides: $$\Large \text{Std Lift}(I→j)=\frac{\text{Lift}(I→j)-\frac{\max{\{P(I)+P(j)-1, 1/n\}}}{P(I)P(j)}}{\frac{1}{\max{\{P(I),P(j)\}}}- \frac{\max{\{P(I)+P(j)-1, 1/n\}}}{P(I)P(j)}}$$ where $n$ is the number of baskets.

In [13]:
lift_threshold = 0.2 # as per the assignment

In [14]:
def std_lift(lift, probX, probY, baskets):
  numerator = lift - (max([probX + probY - 1, 1/baskets])/ (probX * probY))
  denominator = (1/max([probX, probY])) - (max([probX + probY - 1, 1/baskets])/(probX * probY))
  return (numerator / denominator)

With this we incrementally compute the different metrics for the association rules: the confidence, then the interest, the lift, and lastly the standardized lift. This is not the same order as in the results files.

For some metrics we also need the probability of $X$, $P(X)$, and the probability of $X$ and $Y$, $P(Y\cap X)$. For this we simply divide each item or pair counts by their respective total counts, as shown below. These metrics are all computed accordingly to the equations provided in class.

In [15]:
# Compute probabilities of each item and pair being 'drawn'
item_probability = frequent_items.map(lambda item: (item[0], item[1] / total_singles)).collect()
pair_probability = frequent_pairs.map(lambda item: (item[0], item[1] / total_pairs)).collect()

# Collect to lift to iterate over
frequent_items_list = frequent_items.collect()

# Confidence
XY_confidence = frequent_pairs.flatMap(lambda pair: [(X, tuple(set(pair[0]) - set((X,)))[0], pair[1] / supportX) 
                                                     for X, supportX in frequent_items_list 
                                                     if X in pair[0]]
                                       )

# Interest
XY_interest = XY_confidence.flatMap(lambda pair: [((pair[0]), pair[1], pair[2] - probY) 
                                                  for Y, probY in item_probability 
                                                  if Y == pair[1]]
                                    )

# Lift
XY_lift = XY_confidence.flatMap(lambda pair: [((pair[0]), pair[1], pair[2] / probY) 
                                              for Y, probY in item_probability 
                                              if Y == pair[1]])

# Std Lift
XY_stdLift = XY_lift.flatMap(lambda pair: [((pair[0]), pair[1], std_lift(pair[2], probX, probY, total_pairs)) 
                                           for X, probX in item_probability for Y, probY in item_probability 
                                           if X == pair[0] and Y == pair[1] and X != Y]
                             )

Now we can join each RDD as a DataFrame on the same $X$ and $Y$ as shown below. This will enable us to easily sort and filter using the *std_lift* column. As we have a large data set, we decided on leaving every decimal in the results, but we could easily reduce the size of each number by using the function *round* or with string formatting as it was done to align the strings when saving to text files.

In [16]:
# Convert RDDs to DataFrames and join them on columns 'X' and 'Y'
XY_results = (XY_confidence.toDF(['X', 'Y', 'confidence'])
            .join(XY_interest.toDF(['X', 'Y', 'interest']), on = ['X', 'Y'])
            .join(XY_lift.toDF(['X', 'Y', 'lift']), on = ['X', 'Y'])
            .join(XY_stdLift.toDF(['X', 'Y', 'std_lift']), on = ['X', 'Y'])
            )

XY_results.show()

+---------+---------+--------------------+-------------------+------------------+-------------------+
|        X|        Y|          confidence|           interest|              lift|           std_lift|
+---------+---------+--------------------+-------------------+------------------+-------------------+
|271737000| 15777000|    0.81372758686672| 0.7651289660873891|16.743841158817425| 0.8161549645249677|
| 15777000| 44465007| 0.08957566007648561|0.07337159346449179| 5.527974071038692| 0.2686501058155572|
|239873007| 15777000|  0.5051967213114754|0.45659810053214456|10.395289273853983| 0.5051943500163413|
|444814009| 82423001|0.048610261457031144|0.04106990731249045| 6.446681485408158| 0.6648943507275152|
| 44465007|195662009| 0.45988589615534375| 0.3879179932047245| 6.390152794515829|0.45988455962823005|
|195662009| 15777000| 0.30918519817340456| 0.2605865773940737|  6.36201590117763| 0.4578604956427334|
|271737000| 68496003|  0.1280967549497428|0.11635745473202525|10.911787974926542| 

Now we remove any rule below the standardized lift threshold, and then order them from highest standardized lift to lowest. We can notice that the first values have a value slightly over $1$, but it can be assumed that this is caused by the very large data set, and that it should be considered as simply $1$.

In [17]:
# Select std_lift column and filter using those values
XY_results_filtered = XY_results.filter(XY_results.std_lift >= lift_threshold)

# Sort in descending order
XY_results_filtered = XY_results_filtered.sort(XY_results_filtered.std_lift.desc())

XY_rules = XY_results_filtered.collect()

XY_results_filtered.show()

+--------------+--------------+--------------------+--------------------+------------------+------------------+
|             X|             Y|          confidence|            interest|              lift|          std_lift|
+--------------+--------------+--------------------+--------------------+------------------+------------------+
|     302870006| 1551000119108|  0.1540293715127908| 0.15242388809768823| 95.93955942730814|1.0000000000000002|
|     302870006|      80394007| 0.46354879460995896|    0.45871711936174| 95.93955942730814|1.0000000000000002|
|     127013003|90781000119102|  0.1404547949955214| 0.13978804959775862|210.65731457135428|1.0000000000000002|
|     302870006| 1501000119109|0.039938414569954735|  0.0395221273281917| 95.93955942730814|1.0000000000000002|
|     254637007|     424132000|   0.999686299113795|  0.9979378926983903| 571.7699788251903|1.0000000000000002|
|     302870006|97331000119101| 0.05646647015475313|0.055877907204372516| 95.93955942730814|1.0000000000

The results are now sorted and organized and we can save them to a text file. We will save using the codes and also the condition description. This is where the code_to_desc dictionary comes in handy. A lot of string formatting was tested in order to obtain the most readable printing possible.

In [18]:
with open('XY_rules_with_codes.txt', 'w') as f:
  f.write(f"{'X':<15} {'->':<5} {'Y':<15}\t{'Std Lift':<20}\t{'Lift':<20}\t{'Confidence':<20}\t{'Interest':<20}\n")
  for rule in XY_rules:
    f.write(f"{rule['X']:<15} {'->':<5} {rule['Y']:<15}\t{rule['std_lift']:<20}\t{rule['lift']:<20}\t{rule['confidence']:<20}\t{rule['interest']:<20}\n")

In [19]:
with open('XY_rules_with_description.txt', 'w') as f:
  f.write(f"{'X':<80} {'->':<5} {'Y':<80}\t{'Std Lift':<20}\t{'Lift':<20}\t{'Confidence':<20}\t{'Interest':<20}\n")
  for rule in XY_rules:
    f.write(f"{code_to_desc[str(rule['X'])]:<80} {'->':<5} {(code_to_desc[str(rule['Y'])]):<80}\t{rule['std_lift']:<20}\t{rule['lift']:<20}\t{rule['confidence']:<20}\t{rule['interest']:<20}\n")

The association rules of the form $\{X\}\rightarrow\{Y\}$ were obtained, so we can now find rules of the form $\{X,Y\}\rightarrow\{Z\}$. Here we do the same as previously but using both frequent items and frequent pairs. We need the frequent items for the probability of our $Z$ or $j$ in the class equations.

In [20]:
frequent_pairs_list = frequent_pairs.collect()

# Confidence
XYZ_confidence = frequent_triplets.flatMap(lambda item: [((X, Y), tuple(set(item[0]) - set((X, Y)))[0], item[1] / supportPair) 
                                                         for (X, Y), supportPair in frequent_pairs_list 
                                                         if X in item[0] and Y in item[0]]
                                           )

# Interest
XYZ_interest = XYZ_confidence.flatMap(lambda item: [(item[0], item[1], item[2] - probZ) 
                                                    for Z, probZ in item_probability 
                                                    if Z == item[1]]
                                      )

# Lift
XYZ_lift = XYZ_confidence.flatMap(lambda item: [(item[0], item[1], item[2] / probZ) 
                                                for Z, probZ in item_probability 
                                                if Z == item[1]]
                                  )

# Std Lift
XYZ_stdLift = XYZ_lift.flatMap(lambda item: [(item[0], item[1], std_lift(item[2], probPair, probZ, total_triplets)) 
                                             for (X, Y), probPair in pair_probability for Z, probZ in item_probability 
                                             if X in item[0] and Y in item[0] and Z == item[1] and X != Z and Y != Z]
                               )

Group RDDs into a single DataFrame and print first rows. Then filter and sort using the standardized lift of $0.2$ again.

In [21]:
# Convert RDDs to DataFrames and join them on columns 'X' and 'Y'
XYZ_results = (XYZ_confidence.toDF(['(X, Y)', 'Z', 'confidence'])
            .join(XYZ_interest.toDF(['(X, Y)', 'Z', 'interest']), on = ['(X, Y)', 'Z'])
            .join(XYZ_lift.toDF(['(X, Y)', 'Z', 'lift']), on = ['(X, Y)', 'Z'])
            .join(XYZ_stdLift.toDF(['(X, Y)', 'Z', 'std_lift']), on = ['(X, Y)', 'Z'])
            )

XYZ_results.show()

+--------------------+---------+-------------------+-------------------+------------------+-------------------+
|              (X, Y)|        Z|         confidence|           interest|              lift|           std_lift|
+--------------------+---------+-------------------+-------------------+------------------+-------------------+
|{444814009, 19566...| 10509002|  0.404986454280651| 0.3416867901497017| 6.397924220306247|0.40498563604418664|
|{271737000, 15777...| 10509002|0.39944532049686005|0.33614565636591076| 6.310386097318297| 0.3994443390653199|
|{444814009, 16286...| 15777000|0.28300903975194003|0.23441041897260917|  5.82339653293833| 0.2830076500283272|
|{271737000, 10509...|195662009|0.45665622690642926|   0.38468832395581| 6.345276271559053|0.45665441974520743|
|{302870006, 23760...| 40055000| 0.2966597217554943|   0.26233639289502| 8.643092951777122| 0.2966552539578859|
|{162864005, 68496...|195662009| 0.4528352593388874|0.38086735638826813|  6.29218360926259|0.45282946837

In [22]:
# Select std_lift column and filter using those values
XYZ_results_filtered = XYZ_results.filter(XYZ_results.std_lift >= lift_threshold)

# Sort in descending order
XYZ_results_filtered = XYZ_results_filtered.sort(XYZ_results_filtered.std_lift.desc())

XYZ_rules = XYZ_results_filtered.collect()

XYZ_results_filtered.show()

+--------------------+---------+----------+------------------+------------------+--------+
|              (X, Y)|        Z|confidence|          interest|              lift|std_lift|
+--------------------+---------+----------+------------------+------------------+--------+
|{162864005, 12861...|703151001|       1.0|0.9941441346910744|170.76895509802543|     1.0|
|{368581000119106,...| 44054006|       1.0|0.9893965398643384| 94.30883760639537|     1.0|
|{713197008, 44481...| 68496003|       1.0|0.9882606997822825| 85.18395317045814|     1.0|
|{195662009, 79586...| 19169002|       1.0|0.9723077771372304|36.111221730214865|     1.0|
|{127013003, 10509...| 44054006|       1.0|0.9893965398643384| 94.30883760639537|     1.0|
|{444814009, 74400...|428251008|       1.0|0.9926024316935731| 135.1795561159216|     1.0|
|{431855005, 44054...|127013003|       1.0|0.9952529538220175|210.65731457135425|     1.0|
|{1551000119108, 2...|422034002|       1.0|0.9971942102742983| 356.4058955807587|     1.0|

And finally, just like for $\{X\}\rightarrow\{Y\}$ we can save the rules to a text file.

In [37]:
with open('XYZ_rules_with_codes.txt', 'w') as f:
  f.write(f"{'X, Y':<40} {'->':<5} {'Z':<15}\t{'Std Lift':<20}\t{'Lift':<20}\t{'Confidence':<20}\t{'Interest':<20}\n")
  for rule in XYZ_rules:
    X, Y = rule['(X, Y)']
    f.write(f"{X:<20}{Y:<20} {'->':<5} {rule['Z']:<15}\t{rule['std_lift']:<20}\t{rule['lift']:<20}\t{rule['confidence']:<20}\t{rule['interest']:<20}\n")

In [39]:
with open('XYZ_rules_with_description.txt', 'w') as f:
  f.write(f"{'X, Y':<180} {'->':<5} {'Z':<80}\t{'Std Lift':<20}\t{'Lift':<20}\t{'Confidence':<20}\t{'Interest':<20}\n")
  for rule in XYZ_rules:
    X, Y = rule['(X, Y)']
    f.write(f"{code_to_desc[str(X)]:<90}{code_to_desc[str(Y)]:<90} {'->':<5} {(code_to_desc[str(rule['Z'])]):<80}\t{rule['std_lift']:<20}\t{rule['lift']:<20}\t{rule['confidence']:<20}\t{rule['interest']:<20}\n")

We can also save the 10 most frequent itemsets for $k=1,2,3$ to a file.

In [63]:
with open('most_frequent_itemsets.txt', 'w') as f:
    # k = 1
    f.write(f"10 most frequent items for k = 1:\n{'Code':<42} {'Conditions':<120} {'Count':<15}\n")
    for item in frequent_items.take(10):
        f.write(f'{item[0]:<42} {code_to_desc[str(item[0])]:<120} {item[1]:<15}\n')
    # k = 2
    f.write(f"\n10 most frequent items for k = 2:\n{'Code':<42} {'Conditions':<120} {'Count':<15}\n")
    for pair in most_frequent_pairs:
        f.write(f'{pair[0][0]:<14}{pair[0][1]:<28} {code_to_desc[str(pair[0][0])]:<60}{code_to_desc[str(pair[0][1])]:<60} {pair[1]:<15}\n')
    # k = 3
    f.write(f"\n10 most frequent items for k = 3:\n{'Code':<42} {'Conditions':<120} {'Count':<15}\n")
    for triplet in most_frequent_triplets:
        f.write(f"""{triplet[0][0]:<14}{triplet[0][1]:<14}{triplet[0][1]:<14} {code_to_desc[str(triplet[0][0])]:<40}{code_to_desc[str(triplet[0][1])]:<40}{code_to_desc[str(triplet[0][2])]:<40} {triplet[1]:<15}\n""")